In [1]:
import sys, os
from pathlib import Path
sys.path.append(os.path.abspath('./..'))
import utils

import numpy as np
import pandas as pd
from tqdm.auto import tqdm

gdrive = Path('/Users/psr/Library/CloudStorage/GoogleDrive-paru@stanford.edu')
datadir = gdrive / 'My Drive/NMBL Lab/opencap_nmd/data/2023-05_dhd'
# datadir = gdrive / 'Shared drives/OpencapNMD/Data'

# df_session_now = pd.read_excel(datadir / 'session_info.xlsx')
# df_trial_now = pd.read_excel(datadir / 'trial_info.xlsx')
# df_part_now = pd.read_excel(datadir / 'participant_info.xlsx')

# df_part_now.date = pd.to_datetime(df_part_now.date)
# df_part_now.date = df_part_now.date.dt.strftime('%Y-%m-%d')


In [2]:
sessions = utils.getUserSessions()


In [5]:
part_labels = ['type', 'clinician', 'weight', 'height', 'age', 'sex',
               'gender', 'data_sharing', 'orthotics', 'clock_a', 'brooke',
               'time_10mwt', 'time_10mwrt', 'time_tug_line', 'time_tug_cone',
               'clock_b', 'time_5xsts', 'clock_c', 'time_stairs_up', 'notes_p']


In [29]:
# FOR CLINIC DATA

meta = [s for s in sessions]
session_data = []
part_data = []
trial_data = []
for s in tqdm(meta, leave=False):
    sid = s['id']
    pid = s['name']
    session_data.append(dict(
        sid=sid,
        pid=pid,
        created_at=s['created_at'],
        server=s['server'],
        fps=s['meta']['settings']['framerate'] if 'settings' in s['meta'].keys() else np.nan,
        notes_s=np.nan,
    ))
    for t in s['trials']:
        trial = t['name']
        if trial in ('neutral', 'calibration'):
            continue
        trial_data.append(dict(
            sid=sid,
            pid=pid,
            created_at=t['created_at'],
            trial=trial,
            trial_clean=np.nan,
            notes_t=np.nan,
        ))
    tz = 'America/New_York' if pid.startswith('mdf_') else 'America/Los_Angeles'
    part = dict(
        pid=pid,
        date=pd.to_datetime(t['created_at']).tz_convert(tz).strftime('%Y-%m-%d')
    )
    part.update({pl:'' for pl in part_labels})
    part_data.append(part)
        
df_session = pd.DataFrame(session_data).sort_values('pid')
df_part = pd.DataFrame(part_data).sort_values('pid').drop_duplicates()
df_trial = pd.DataFrame(trial_data).sort_values(['pid', 'trial'])

df_part.shape, df_session.shape, df_trial.shape


df_part.to_excel('participant_info_clinic.xlsx')
df_trial.to_excel('trial_info_clinic.xlsx')
df_session.to_excel('session_info_clinic.xlsx')


  0%|          | 0/166 [00:00<?, ?it/s]

In [26]:
df_part.pid.unique()


array(['3d17b3ee', '58385b9a', '6ec44398', '7e6bd5de', '9473a88c',
       '9bda23b6', 'CONTROL005', 'CPET_001', 'DB-039', 'DB-139', 'DB-148',
       'DB-157', 'DB-182', 'DB-183', 'DB-184', 'DB-185', 'DB-238',
       'DB-245', 'DB-251', 'DB-251_stairs', 'DB-258', 'DB-267', 'DB-275',
       'DB-287', 'DB-305', 'DB-310', 'DB-318', 'DB-333', 'DB-334',
       'DB-361', 'DB-369', 'DB-493', 'DB157', 'DB180', 'DB207', 'DB237',
       'DB238', 'DB262', 'DB302', 'DB305', 'DB310', 'DB359', 'DB_162',
       'DB_259', 'DB_263', 'DIGBI160', 'DIGBI_01DEC2022',
       'DIGBI_160_stairs', 'DM1_001', 'DM1_002', 'DM1_003', 'DM1_004',
       'DM1_005', 'DM1_007', 'DM1_008', 'DM1_009', 'DM1_010', 'DM1_011',
       'DM1_013', 'DM1_014', 'DM1_015', 'DM1_016', 'DM1_017', 'DM1_018',
       'DM1_019', 'DM1_020', 'DM1_021', 'DM1_022', 'DM1_023', 'DM1_024',
       'DM1_026', 'DM1_027', 'DM1_028', 'DM1_25', 'DM2_001', 'DM2_002',
       'DMD_002', 'FSHD_01', 'FSHD_02', 'NoID_002_22JAN2022',
       'NoID_002_22JAN23

In [9]:
def get_pid(s):
    name = s['name'].lower()
    if len(name) == 4 and name[1:].isnumeric() and name[0]=='p':
        if int(name[1:]) < 10:
            return ''
        return name
    if name[:4] == 'mdf_' and name[4:7].isnumeric():
        return name
    return name

meta = [s for s in sessions if get_pid(s)]
session_data = []
part_data = []
trial_data = []
for s in tqdm(meta, leave=False):
    sid = s['id']
    pid = get_pid(s)
    # if int(pid[1:]) < 10:
    #     continue
    session_data.append(dict(
        sid=sid,
        pid=pid,
        created_at=s['created_at'],
        server=s['server'],
        fps=s['meta']['settings']['framerate'],
        notes_s=np.nan,
    ))
    for t in s['trials']:
        trial = t['name']
        if trial in ('neutral', 'calibration'):
            continue
        trial_data.append(dict(
            sid=sid,
            pid=pid,
            created_at=t['created_at'],
            trial=trial,
            trial_clean=np.nan,
            notes_t=np.nan,
        ))
    tz = 'America/Los_Angeles' if pid[0] == 'p' else 'America/New_York'
    part = dict(
        pid=pid,
        date=pd.to_datetime(t['created_at']).tz_convert(tz).strftime('%Y-%m-%d')
    )
    part.update({pl:'' for pl in part_labels})
    part_data.append(part)
        
df_session = pd.DataFrame(session_data).sort_values('pid')
df_part = pd.DataFrame(part_data).sort_values('pid').drop_duplicates()
df_trial = pd.DataFrame(trial_data).sort_values(['pid', 'trial'])

df_part.shape, df_session.shape, df_trial.shape


  0%|          | 0/164 [00:00<?, ?it/s]

KeyError: 'settings'

In [ ]:
df_part.pid.unique()

In [ ]:
df_part_new = df_part_now.set_index(['pid', 'date']).combine_first(df_part.set_index(['pid', 'date']))
df_part_new.reset_index(inplace=True)
df_part_new.sort_values(['date', 'pid'], inplace=True)


In [ ]:
df_session_new = df_session_now.set_index('sid').combine_first(df_session.set_index('sid'))
cols = ['pid', 'created_at']
cols += [x for x in df_session_new.columns if x not in cols]
cols = [x for x in cols if x != 'notes_s'] + ['notes_s']
df_session_new = df_session_new[cols]
df_session_new.sort_values('pid', inplace=True)
df_session_new.reset_index(inplace=True)


In [ ]:
df_trial_new = df_trial_now.set_index(['sid', 'trial']).combine_first(df_trial.set_index(['sid', 'trial']))
df_trial_new.reset_index(inplace=True)
df_trial_new.sort_values(['pid', 'created_at'], inplace=True)
cols = ['pid', 'sid', 'created_at', 'trial', 'trial_clean']
cols += [x for x in df_trial_new.columns if x not in cols]
cols = [x for x in cols if x != 'notes_t'] + ['notes_t']
df_trial_new = df_trial_new[cols]
# df_trial_new


In [ ]:
df_part_new


In [ ]:
df_part_new.to_excel(datadir / 'temp/participant_info.xlsx', index=False)
df_session_new.to_excel(datadir / 'temp/session_info.xlsx', index=False)
df_trial_new.to_excel(datadir / 'temp/trial_info.xlsx', index=False)
